In [1]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
alt.themes.enable('report')

ThemeRegistry.enable('report')

## Growth Line Chart

In [2]:
df = pd.read_excel("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/data/raw/regionalgrossvalueaddedbalancedperheadandincomecomponents-5.xlsx", sheet_name='Table 5', skiprows=1)
df = df.query("ITL=='ITL1'")
df = df.drop(columns=['ITL'])
df = df.melt(id_vars=['ITL code', 'Region name'], var_name='date', value_name='value')
df.columns = ['id', 'name', 'date', 'value']
df['value'] = pd.to_numeric(df['value'], errors='coerce')/100
df = df.dropna()

base = alt.Chart(df).encode(
    x=alt.X('date:T', title='Year'),
    y=alt.Y('value:Q', title='Value'),
    color=alt.Color('name:N', title='Region')
)

lines = base.mark_line().encode(
    detail='id:N'
)

lines

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# Cumulative

In [5]:
df = pd.read_excel("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/data/raw/regionalgrossvalueaddedbalancedperheadandincomecomponents-5.xlsx", sheet_name='Table 1', skiprows=1)
df = df.query("ITL=='ITL1'")
df = df.drop(columns=['ITL'])
df = df[['ITL code', 'Region name'] + [str(i) for i in range(2000, 2023)]]
df = df.melt(id_vars=['ITL code', 'Region name'], var_name='date', value_name='nominal')
gva_pc_df = df.copy()


## and get the deflators
df = pd.read_excel("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/data/raw/rgva_itl1_itl2_w_deflators.xlsx", sheet_name='Table 1d', skiprows=1)
df = df[df['SIC07 description'] == 'All industries']
df = df[['ITL code', 'Region name'] + [str(i) for i in range(2000, 2023)]]
temp_df = df.copy()
for i in range(2000, 2023):
    df[str(i)] = df[str(i)]/temp_df['2000']
df = df.melt(id_vars=['ITL code', 'Region name'], var_name='date', value_name='deflator')
deflators = df.copy()

df = pd.merge(gva_pc_df, deflators, on=['ITL code', 'Region name', 'date'], how='left')
df['value'] = df['nominal']/df['deflator']

# now get growth since 2000
df = df[['ITL code', 'Region name', 'date', 'value']]
df = df.pivot(index=['ITL code', 'Region name'], columns='date', values='value').reset_index()
temp_df = df.copy()
for i in range(2000, 2023):
    df[str(i)] = df[str(i)]/temp_df['2000']
df = df.melt(id_vars=['ITL code', 'Region name'], var_name='date', value_name='value')

df['label'] = np.where(df['date'] == '2022', df['Region name'], '')

base = alt.Chart(df).encode(
    x=alt.X('date:T', title='Year'),
    y=alt.Y('value:Q', title='', scale=alt.Scale(zero=False), axis=alt.Axis(labelExpr="datum.value + '%'")),
    color=alt.Color('Region name:N', title='Region', scale=alt.Scale(scheme='category20'), legend=None)
)

lines = base.mark_line().encode(
    detail='ITL code:N'
)

labels = base.mark_text(align='left', dx=5).encode(
    text='label:N'
)

chart = lines + labels

chart.save("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/charts/gva_pc_growth.json")
chart.save("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/charts/gva_pc_growth.png", scale_factor=2.0)

chart




/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [7]:
df.to_csv("/Users/finn/Documents/GitHub/GrowthDiagnostics/report_figures/data/clean/gva_pc_growth.csv", index=False)

In [ ]:

df = df.melt(id_vars=['ITL code', 'Region name'], var_name='date', value_name='value')
df.columns = ['id', 'name', 'date', 'value']
df['value'] = pd.to_numeric(df['value'], errors='coerce')/100
df = df.dropna()

base = alt.Chart(df).encode(
    x=alt.X('date:T', title='Year'),
    y=alt.Y('value:Q', title='Value'),
    color=alt.Color('name:N', title='Region')
)

lines = base.mark_line().encode(
    detail='id:N'
)

lines

In [3]:
df

,"Table 4: Gross value added (balanced) annual growth rates, percentage",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,ITL,ITL code,Region name,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
1,UK,UK,United Kingdom,4.0,5.4,4.3,4.1,5.8,4.9,5.9,...,3.0,3.9,4.4,3.2,4.0,4.4,3.5,3.9,-4.8,7.2
2,Other,TLB,England,4.2,4.8,4.4,4.4,5.7,4.7,5.8,...,3.3,4.1,4.9,3.8,4.2,4.2,3.3,4.0,-4.5,7.1
3,ITL1,TLC,North East,1.5,3.0,4.1,6.8,5.9,8.0,4.7,...,2.4,1.4,3.1,3.7,1.5,2.5,1.7,4.6,-5.8,7.0
4,ITL2,TLC1,Tees Valley and Durham,4.1,3.3,1.7,5.3,7.3,6.9,3.7,...,1.8,2.1,2.5,3.6,0.4,0.8,3.6,3.7,-4.1,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,ITL3,TLN0E,Lisburn and Castlereagh,5.6,7.8,5.9,2.7,7.2,2.9,6.1,...,6.1,2.9,3.8,5.6,6.1,2.7,6.4,7.4,-2.9,5.5
233,ITL3,TLN0F,Mid and East Antrim,4.6,9.4,-4.0,3.2,2.9,6.5,-0.1,...,13.4,-1.4,-2.9,12.1,-12.1,-3.6,-13.9,5.9,-2.0,5.6
234,ITL3,TLN0G,Fermanagh and Omagh,5.1,6.6,4.6,3.8,7.9,5.5,7.7,...,6.5,-3.9,6.8,1.1,6.3,5.7,5.4,5.3,1.6,7.2
235,Other,TLZ,Extra-Regio,12.2,50.1,-0.8,-11.7,2.3,7.3,17.0,...,-10.9,-6.8,-18.9,-40.4,0.1,40.3,38.8,-5.0,-35.8,19.1
